# Breadth-depth dilemma

Notebook that goes through the reproduction of figures for the paper "Sampling few options is the optimal solution to the breadth-depth dilemma in a finite sample capacity model of decision-making". All the functions used are in "Functions.jl" Julia module.

In [ ]:
push!(LOAD_PATH, "./");

In [ ]:
using PyPlot, CurveFit, Functions

## Figure 1
### Panel (a)

In [ ]:
figure(figsize=(10,6))
C = collect(20:40:140)
for j in 1:length(C)
    M = divisors(C[j])
    U = zeros(length(M))
    for i in 1:length(M)
        U[i] = uni_utility_flat(C[j],M[i])
    end
    semilogx(M,U,"-o",label=string("C = ",C[j]))
end
legend()
ylabel("Utility")
xlabel("M, sampled alternatives");
#savefig("fig2a.png")

In [ ]:
C = [1,2,3,4,5,6,7,10,50,100,500,1000,5000,10000,50000]
max_U = zeros(length(C))
M_opt = zeros(length(C))
max_U_rich = zeros(length(C))
M_opt_rich = zeros(length(C))
max_U_poor = zeros(length(C))
M_opt_poor = zeros(length(C))
α_poor = 1
β_poor = 3
α_rich = 3
β_rich = 1
for j in 1:length(C)
    M = divisors(C[j])
    U_rich = zeros(length(M))
    U_poor = zeros(length(M))
    U = zeros(length(M))
    for i in 1:length(M)
        U[i] = uni_utility_flat(C[j],M[i])
        U_rich[i] = uni_utility(C[j],M[i],α_rich,β_rich)
        U_poor[i] = uni_utility(C[j],M[i],α_poor,β_poor)
    end
    max_U[j] = findmax(U)[1]
    M_opt[j] = M[findmax(U)[2]]
    max_U_rich[j] = findmax(U_rich)[1]
    M_opt_rich[j] = M[findmax(U_rich)[2]]
    max_U_poor[j] = findmax(U_poor)[1]
    M_opt_poor[j] = M[findmax(U_poor)[2]]
end

### Panel (b)

In [ ]:
figure(figsize=(10,6))
loglog(C,M_opt,"-o")
b1,m1=linear_fit(log.(C[1:5]),log.(M_opt[1:5]))
b2,m2=linear_fit(log.(C[8:end]),log.(M_opt[8:end]))
M_fitted1 = exp.(log.(C[1:end]).*m1 .+ b1)
M_fitted2 = exp.(log.(C[8:end]).*m2 .+ b2)
loglog(C[1:end],M_fitted1,"--",label=string("m = ",round(m1,digits=2)))
loglog(C[8:end],M_fitted2,"--",label=string("m = ",round(m2,digits=2)))
legend()
ylabel("Optimal M")
xlabel("Capacity");
#savefig("fig2b.png")

### Panel (c)-(d)

In [ ]:
figure(figsize=(10,6))
semilogx(C,M_opt./C,"-o")
semilogx(C,M_opt_rich./C,"-o",label="rich")
semilogx(C,M_opt./C,"-o",label="flat")
semilogx(C,M_opt_poor./C,"-o",label="poor")
legend()
#xlim([0,64])
ylabel("Optimal M/C")
xlabel("Capacity")
ylabel("Optimal M/C")
xlabel("Capacity");

### Fits for non-flat priors

In [ ]:
figure(figsize=(10,5))
loglog(C,M_opt_rich,"-o",label="rich")
loglog(C,M_opt_poor,"-o",label="poor")
b1,m1=linear_fit(log.(C[8:end]),log.(M_opt_rich[8:end]))
b2,m2=linear_fit(log.(C[8:end]),log.(M_opt_poor[8:end]))
M_fitted1 = exp.(log.(C[8:end]).*m1 .+ b1)
M_fitted2 = exp.(log.(C[8:end]).*m2 .+ b2)
loglog(C[8:end],M_fitted1,"--",label=string("m = ",round(m1,digits=2)))
loglog(C[8:end],M_fitted2,"--",label=string("m = ",round(m2,digits=2)))
legend()
ylabel("Optimal M")
xlabel("Capacity");

## Figure 3

### Panel (a), flat prior

In [ ]:
α,β = 1,1
#Adjustable number of samples to make the code run faster, 1E6 is good enough
utility_precision = 1E6
Ls = Any[]
us_max = Any[]
push!(Ls,[1])
push!(us_max,0.5)
for j in 2:20
    if j<=7
        L = Int64.(ones(j))
    else
        M = Int64(round(sqrt(j)))
        L = zeros(Int64,j)
        l_i = Int64(floor(j/M))
        for i in 1:M
            L[i] = l_i
        end
        L[M+1]=j-l_i*M
    end
    push!(L,0)
    #Adjust 20*j to explore more or less distributions
    L_opt,u_max = dist_max(α,β,L,20*j,utility_precision)
    pop!(L_opt)
    push!(Ls,L_opt)
    push!(us_max,u_max)
end

In [ ]:
figure(figsize=(10,6))
subplots_adjust(wspace=0.5,hspace=0.3)
for i in 1:20
subplot(4,5,i)
    annotate("C = $(i)",
	xy=[1;0.6],
	xycoords="axes fraction",
	xytext=[-10,10],
	textcoords="offset points",
	fontsize=10.0,
	ha="right",
	va="bottom")
    yticks([0,1,2,3,4,5])
    if i < 9
    xticks([1:1:i;])
    else
        xti=[5:5:i+1;]
        pushfirst!(xti,1)
        xticks(xti)
    end
    x = [1:1:i;]
    bar(x,Ls[i])
end
#savefig("./figures/fig3a.png")

### Panel (b)-(c), non-flat priors

In [ ]:
α_poor = 1
β_poor = 3
α_rich = 3
β_rich = 1
#Adjustable number of samples to make the code run faster, 1E6 is good enough
utility_precision = 1E6
Ls_rich = Any[]
Ls_poor = Any[]
us_max_rich = Any[]
us_max_poor = Any[]
for j in 2:20
    if j<=7
        L = Int64.(ones(j))
    else
        M = Int64(round(sqrt(j)))
        L = zeros(Int64,j)
        l_i = Int64(floor(j/M))
        for i in 1:M
            L[i] = l_i
        end
        L[M+1]=j-l_i*M
    end
    #Adjust 20*j to explore more or less distributions
    L_opt_rich,u_max_rich = dist_max(α_rich,β_rich,L,20*j,utility_precision)
    L_opt_poor,u_max_poor = dist_max(α_poor,β_poor,L,20*j,utility_precision)
    push!(Ls_rich,L_opt_rich)
    push!(Ls_poor,L_opt_poor)
    push!(us_max_rich,u_max_rich)
    push!(us_max_poor,u_max_poor)
end

In [ ]:
pushfirst!(Ls_rich,[1])
figure(figsize=(10,6))
subplots_adjust(wspace=0.5,hspace=0.3)
for i in 1:20
subplot(4,5,i)
    annotate("C = $(i)",
	xy=[1;0.6],
	xycoords="axes fraction",
	xytext=[-10,10],
	textcoords="offset points",
	fontsize=10.0,
	ha="right",
	va="bottom")
    yticks([0,1,2,3,4,5])
    if i < 9
    xticks([1:1:i;])
    else
        xti=[5:5:i+1;]
        pushfirst!(xti,1)
        xticks(xti)
    end
    x = [1:1:i;]
    bar(x,Ls_rich[i])
end
#savefig("./figures/fig3b.png")

In [ ]:
pushfirst!(Ls_poor,[1])
figure(figsize=(10,6))
subplots_adjust(wspace=0.5,hspace=0.3)
for i in 1:20
subplot(4,5,i)
    annotate("C = $(i)",
	xy=[1;0.6],
	xycoords="axes fraction",
	xytext=[-10,10],
	textcoords="offset points",
	fontsize=10.0,
	ha="right",
	va="bottom")
    yticks([0,1,2,3,4,5])
    if i < 9
    xticks([1:1:i;])
    else
        xti=[5:5:i+1;]
        pushfirst!(xti,1)
        xticks(xti)
    end
    x = [1:1:i;]
    bar(x,Ls_poor[i])
end
#savefig("./figures/fig3c.png")

## Figure 4

In [ ]:
#This one takes time! To get a rough estimate, decrease the precision below, or the number of perturbations "sample"
C = [2,3,4,5,6,7,8,9,10,20,50,100,200,500]
α_poor = 1
β_poor = 3
α_rich = 3
β_rich = 1
#Adjustable number of samples to make the code run faster
utility_precision = 1E6
U_uni_flat = zeros(length(C))
U_uni_rich = zeros(length(C))
U_uni_poor = zeros(length(C))
U_optimal_flat = zeros(length(C))
U_optimal_rich = zeros(length(C))
U_optimal_poor = zeros(length(C))
Ls_flat = Any[]
Ls_rich = Any[]
Ls_poor = Any[]
U_optimal_flat = Any[]
U_optimal_rich = Any[]
U_optimal_poor = Any[]
M_opt_flat = zeros(length(C))
M_opt_rich = zeros(length(C))
M_opt_poor = zeros(length(C))
for j in 1:length(C)
    M = divisors(C[j])
    #Square-root law heuristic
    if C[j]<=7
        L = Int64.(ones(C[j]))
    else
        M_o = Int64(floor(sqrt(C[j])))
        L = zeros(Int64,C[j])
        for i in 1:M_o
             L[i] = M_o
        end
        L[M_o+1]=C[j]-M_o*M_o
    end
    #Estimate the estimate of utility for square-root distributions with following precision
    uni_prec = 1E6
    U_uni_flat[j]=utility_estimate(L,1,1,uni_prec)
    U_uni_poor[j]=utility_estimate(L,α_poor,β_poor,uni_prec)
    U_uni_rich[j]=utility_estimate(L,α_rich,β_rich,uni_prec)
    #Setting the number of perturbations proposed
    if C[j] >= 100
        sample = 2500
    else
        sample = 50*C[j]
    end
    #Optimal distribution
    L_opt_rich,u_max_rich = dist_max(α_rich,β_rich,L,sample,utility_precision)
    L_opt_poor,u_max_poor = dist_max(α_poor,β_poor,L,sample,utility_precision)
    L_opt_flat,u_max_flat = dist_max(1,1,L,sample,utility_precision)
    push!(Ls_flat,L_opt_flat)
    push!(Ls_rich,L_opt_rich)
    push!(Ls_poor,L_opt_poor)
    push!(U_optimal_flat,u_max_flat)
    push!(U_optimal_rich,u_max_rich)
    push!(U_optimal_poor,u_max_poor)
end

### Panel (a)

In [ ]:
semilogx(C,((U_optimal_flat-U_uni_flat)./(U_uni_flat)).*100,"-",label="Flat")
semilogx(C,((U_optimal_rich-U_uni_rich)./(U_uni_rich)).*100,"-",label="rich")
semilogx(C,((U_optimal_poor-U_uni_poor)./(U_uni_poor)).*100,"-",label="poor")
xticks(C)
yticks([-5,0,5,10])
legend()
xlabel("Capacity")
ylabel("% improvement");
#savefig("./figures/fig4a.png")

### Panel (b)

In [ ]:
frac_sampled_flat = zeros(length(C))
frac_sampled_rich = zeros(length(C))
frac_sampled_poor = zeros(length(C))
for j in 1:length(C)
    for i in 1:length(Ls_flat[j])
        if Ls_flat[j][i] > 0
            frac_sampled_flat[j] += 1
        end
        if Ls_poor[j][i] > 0
            frac_sampled_poor[j] += 1
        end
        if Ls_rich[j][i] > 0
            frac_sampled_rich[j] += 1
        end
    end
end

In [ ]:
semilogx(C,frac_sampled_flat./C,"o-",label="Flat")
semilogx(C,frac_sampled_rich./C,"o-",label="Rich")
semilogx(C,frac_sampled_poor./C,"o-",label="Poor")
xticks(C)
yticks([0,0.5,1])
legend()
xlabel("Capacity")
ylabel("Fraction of sampled alternatives");
#savefig("./figures/fig4b.png")